### Limpieza e inspección de los datos

Este notebook está esturcturado como sigue:

- **Importación de librerias**
- **Bloque A**: carga de los datos.
- **Bloque B**: inspección de los datos.
- **Bloque C**: limpieza y adecuación.
- **Bloque D**: exportación del conjunto resultante.

### Importación librerias

In [2]:
import pandas as pd
import re

import cld3

### BLOQUE A: Carga de los datos

**Pandas**: librería más popular de python que proporciona las herramientas y estructuras necesarias para manipular y analizar datos. La estructura básica de Pandas es el **DataFrame**, una colección ordenada de columnas con nombres y tipos, donde una sola fila representa un único caso (observación) y las columnas representan atributos particulares.

Guía: https://pandas.pydata.org/docs/getting_started/index.html

In [60]:
# Leemos el CSV que contiene los datos y lo cargamos en memoria
df = pd.read_csv("../data/amazon_reviews.csv")

In [61]:
df.shape

(24000, 3)

### BLOQUE B: inspección de los datos

El objetivo de la inspección es la familiarización con el conjunto de datos. Algunas preguntas iniciales que podría estar bien hacerse pueden ser:

- ¿En qué tipo de objeto están almacenados los datos?
- ¿Cuál es su dimensión?
- ¿Hay datos ausentes?
- ¿Que tipos diferentes de datos hay?
- ...

In [37]:
# Ver el tipo de objeto con el que estamos trabajando
type(df)

pandas.core.frame.DataFrame

In [38]:
# Dimensiones del dataframe
df.shape

(24000, 3)

In [39]:
# Otra manera más interesante de proporcionar la misma información anterior
print(f"Hay un dataframe de {df.shape[0]} filas y {df.shape[1]} columnas.")

Hay un dataframe de 24000 filas y 3 columnas.


In [40]:
# Resumen conciso del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24000 entries, 64522 to 61666
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class_index   24000 non-null  int64 
 1   review_title  24000 non-null  object
 2   review_text   24000 non-null  object
dtypes: int64(1), object(2)
memory usage: 750.0+ KB


In [41]:
# Mostrar las primeras 5 filas
df.head()

,class_index,review_title,review_text
64522,2,Super saver good quality black ink.,"Smooth, good black density ink. For general pu..."
53816,2,Solid.,"Good, solid pair of sunglasses. Ray Ban makes ..."
111956,2,Fantastic Computer Speakers - Best You Can Buy,These speakers have fabulous sound without an ...
73647,1,Pray you won't need the warranty!,We have used Farberware Percolators since the ...
110426,2,Good for Beginners,I'm a fan of Nancy Kress's how-to books for Wr...


In [42]:
# Mostrar las últimas 5 filas
df.tail()

,class_index,review_title,review_text
519,2,Amazing!,"I loved the final cut since I first heard it, ..."
46148,1,WHY BUY TOM TOM? WHEN YOU CAN GET BETTER? A LO...,I am a professional driver. I have compared ma...
35360,2,Good Quad Ramps,Got these for loading my 2 up Can Am 400 into ...
106760,1,Not scary I think Hostel is way better!,I know a lot of people didn't like Hostel but ...
61666,2,Love on the Prarie,I adored this story. It was a true illustratio...


In [43]:
# ¿Cuáles son los posibles valores que puede tomar la variable 'class_index'?
df['class_index'].unique()

array([2, 1])

La variable "class_index" indica el sentimiento de cada reseña:

- 1: sentimiento negativo
- 2: sentimiento positivo

### BLOQUE C: limpieza y adecuación

In [44]:
# Eliminación de las columnas que no son utiles a nuestro estudio
# Nos quedaremos solo con 'Revies'
df = df.drop(columns=['review_title'])
df.head()

,class_index,review_text
64522,2,"Smooth, good black density ink. For general pu..."
53816,2,"Good, solid pair of sunglasses. Ray Ban makes ..."
111956,2,These speakers have fabulous sound without an ...
73647,1,We have used Farberware Percolators since the ...
110426,2,I'm a fan of Nancy Kress's how-to books for Wr...


In [45]:
""" 
Renombrar las columnas:
    - review_text -> text
    - class_index --> sentiment
"""
df.rename(columns={'review_text': 'text', 'class_index':'sentiment'}, inplace=True)

df.head()

,sentiment,text
64522,2,"Smooth, good black density ink. For general pu..."
53816,2,"Good, solid pair of sunglasses. Ray Ban makes ..."
111956,2,These speakers have fabulous sound without an ...
73647,1,We have used Farberware Percolators since the ...
110426,2,I'm a fan of Nancy Kress's how-to books for Wr...


In [46]:
# Calcular la longitud de cada reseña
df['length'] = df['text'].apply(len)

In [47]:
# Resumen de la nueva columna "length"
df['length'].describe()

count    24000.000000
mean       402.431667
std        234.145683
min         18.000000
25%        206.000000
50%        352.000000
75%        562.000000
max       1006.000000
Name: length, dtype: float64

In [48]:
# busqueda de los elementos con longitud mínima (15)
df[df.length == 15]

,sentiment,text,length


In [49]:
# Vamos a remover aquellos textos que tiene más de 565 caracteres (Q3)
df = df[df.length <= 565]

In [50]:
# Comprobamos las dimensiones actuales del dataframe
df.shape

(18079, 3)

In [51]:
# Detección del idioma en el que está escrita cada reseña
df['language'] = df['text'].apply(lambda x: cld3.get_language(x)[0])

In [52]:
# ¿En que idioma están escritas las reseñas? ¿Cuantas reseñas tenemos para cada idioma?
df['language'].value_counts()

en    18022
es       42
fr        4
pt        3
ha        1
ca        1
nl        1
de        1
mg        1
it        1
pl        1
so        1
Name: language, dtype: int64

In [53]:
# ¿Cual son las reseñas en español?
df[df.language=='es']

,sentiment,text,length,language
17295,1,"En español decimos ""voz chillona"" cuando el ti...",471,es
24628,1,Los zapatos son buenos pero escogi mal el tama...,106,es
11234,1,hola escribo para publicar que el diseño de es...,305,es
91093,2,Un micrófono legendario a buen precio. Aunque ...,405,es
94094,2,Este pequeño manual es ya un clasico de muchos...,436,es
109160,2,Pasaron muchos años para que Juan Gabriel saca...,151,es
28814,2,Encontré este disco por casualidad y me result...,319,es
83554,2,¿Porque no era lo que esperaba? Leí Melocotone...,420,es
95537,2,Es una demostracion adicional de lo que es cap...,227,es
48206,2,¿Que tan util es la informacion en este libro?...,439,es


In [54]:
# Nos quedamos solo con aquellas que están escritas únicamente en inglés
df = df[df['language']=='en']

In [55]:
# Comprobamos las dimensiones actuales del dataframe
df.shape

(18022, 4)

### BLOQUE E: Exportación del conjunto resultante

En esta última fase exportamos y guardamos el conjunto de datos ya limpiado, procesado y adecuado en la carpeta 'data'. Solo guardaremos aquellas columnas que posteriormente nos serán útiles.
Ya que hay bastante datos nos vamo

In [56]:
# Eliminamos las columnas que ya no serán necesarias para los siguiente pasos
df = df.drop(columns=['language'])

In [57]:
# muestreo proporcional
df_sample = df.groupby('sentiment', group_keys=False).apply(
    lambda x: x.sample(frac=0.4, random_state=2023)
)

df_sample.head()

,sentiment,text,length
52409,1,I got this product and only paid $10 for it. I...,376
42044,1,great cards but low quality. i spent all my $$...,146
101804,1,I really hated this book. Flyy girl was one of...,553
69916,1,Langston Hughes is definitely a better poet th...,301
94251,1,"Note that, despite being called ""Norpro 422D G...",301


In [58]:
# vamos a combrobar que las proporcionese se hay mantenido iguales
print("Proporciones antes del muestreo:")
print(df['sentiment'].value_counts(normalize=True), "\n")

print("Proporciones despues del muestreo:")
print(df_sample['sentiment'].value_counts(normalize=True))

Proporciones antes del muestreo:
2    0.512096
1    0.487904
Name: sentiment, dtype: float64 

Proporciones despues del muestreo:
2    0.512138
1    0.487862
Name: sentiment, dtype: float64


In [59]:
# Guardamos los datos ya procesados en un fichero csv
df_sample.to_csv('../data/amazon_reviews_clean.csv', index=False)